##  This notebook takes labels from build_training_data/training_images/labels.json and images from build_training_data/training_images/ and uses that to create parking spot car detection models for ParkEz

In [1]:
import os
import json
import time
import numpy as np
import pandas as pd
import cv2
import torch
from torch import nn, optim
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import Lambda

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Convolutional layer 1
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)  
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()
        
        # Convolutional layer 3
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.relu3 = nn.ReLU()

        # Convolutional layer 4
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.relu4 = nn.ReLU()

        # Convolutional layer 5
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()

        # Max pool layer
        self.pool = nn.MaxPool2d(kernel_size=2)

        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

        # Fully connected layers
        self.fc1 = nn.Linear(512 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 2)

    def forward(self, x):
        # Convolutional layer 1
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.pool(out)

        # Convolutional layer 2
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.pool(out)

        # Convolutional layer 3
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.pool(out)

        # Convolutional layer 4
        out = self.conv4(out)
        out = self.bn4(out)
        out = self.relu4(out)
        out = self.pool(out)

        # Convolutional layer 5
        out = self.conv5(out)
        out = self.bn5(out)
        out = self.relu5(out)
        out = self.pool(out)

        # Flatten for fully connected layer
        out = out.view(out.size(0), -1)

        # Fully connected layer 1
        out = self.fc1(out)
        out = self.dropout(out)

        # Fully connected layer 2
        out = self.fc2(out)
        out = self.dropout(out)

        # Fully connected layer 3
        out = self.fc3(out)

        return out


# Originally in Model_Maker notebook, this preps cropped parking spaces for ML processing
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to 256x256
    transforms.ToTensor(),  # Convert to PyTorch tensor
    transforms.Normalize((0.5,0.5,0.5,), (0.5,0.5,0.5,))  # Normalize pixel values in the range [-1, 1]
])

In [3]:
def get_models():
    old_models = {}
    for key in spot_keys:
        model_path = os.path.join('Archive', 'old_models', key + '.pth')
        old_models[key] = CNN()
        old_models[key].load_state_dict(torch.load(model_path)) 
        old_models[key].eval() 
    return old_models

def predict_image(to_predict_image, predicting_model_dict):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ret = {}
    for key in spot_keys:
        dir_name = 'input'
        full_path = os.path.join(dir_name, to_predict_image)
        image = Image.open(full_path)
        spots_file_path = os.path.join('spots.json')
        with open(spots_file_path, 'r') as spots_file:
            spots_data = json.load(spots_file)
        x, x_w, y, y_h = spots_data[key]
        cropped_image = image.crop((x, y, x_w, y_h))
        input_tensor = transform(cropped_image)
        input_tensor = input_tensor.unsqueeze(0)
        
        # Move the input tensor to the same device as the model
        input_tensor = input_tensor.to(device)
        
        with torch.no_grad():
            output = predicting_model_dict[key](input_tensor)
            _, predicted = torch.max(output, 1)

        prediction = predicted.item()
        ret[key] = False
        if prediction == 0: ret[key] = True
    return ret

In [4]:
# !ls lot_image_uploader/baseline_ml_diag
def predict_image(to_predict_image, spots, predicting_model_dict):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ret = {}
    for key in spots:
        full_path = os.path.join('lot_image_uploader', 'baseline_ml_diag', to_predict_image)
        image = Image.open(full_path)
        x, x_w, y, y_h = spots_data[key]
        cropped_image = image.crop((x, y, x_w, y_h))
        input_tensor = transform(cropped_image)
        input_tensor = input_tensor.unsqueeze(0)
        
        # Move the input tensor to the same device as the model
        input_tensor = input_tensor.to(device)
        
        with torch.no_grad():
            output = predicting_model_dict[key](input_tensor)
            _, predicted = torch.max(output, 1)

        prediction = predicted.item()
        ret[key] = False
        if prediction == 0: ret[key] = True
    return ret

In [5]:
def get_models():
    old_models = {}
    for key in ['b2', 'b3']:
        model_path = key + '.pth'
        old_models[key] = CNN()
        # Add map_location='cpu' to load the model onto the CPU
        old_models[key].load_state_dict(torch.load(model_path, map_location='cpu')) 
        old_models[key].eval()
    return old_models

models_es_dict = get_models()

directory_path = 'lot_image_uploader/baseline_ml_diag'
spots_file_path = 'build_training_data/spots.json'
with open(spots_file_path, 'r') as spots_file:
    spots_data = json.load(spots_file)

file_names = os.listdir(directory_path)
for im in file_names:
    print( im + ' ' + str(predict_image(im, ['b2','b3'], models_es_dict)))

colltown_202310291259.jpg {'b2': True, 'b3': True}
colltown_202310290938.jpg {'b2': False, 'b3': True}
colltown_202310291446.jpg {'b2': True, 'b3': True}
colltown_202310291056.jpg {'b2': True, 'b3': True}
colltown_202310291229.jpg {'b2': False, 'b3': False}
colltown_202310291158.jpg {'b2': True, 'b3': True}
colltown_202310291330.jpg {'b2': True, 'b3': True}
colltown_202310291416.jpg {'b2': True, 'b3': True}
colltown_202310291128.jpg {'b2': True, 'b3': True}
colltown_202310291025.jpg {'b2': False, 'b3': True}
